In [1]:
from typing import Dict, List
from datamodel import Order, OrderDepth, TradingState, Trade
import json
import math

class Trader:
    def __init__(self):
        self.mid_prices = {
            "CROISSANTS": [],
            "JAMS": [],
            "DJEMBES": [],
            "PICNIC_BASKET1": []
        }

    POSITION_LIMITS = {
        "CROISSANTS": 250,
        "JAM": 350,
        "PICNIC_BASKET2": 100
    }
    
    def run(self, state: TradingState):
        orders = {}
        conversions = 0  # no conversions in this minimal version
        traderData = "pb2_arbitrage_v1"
        
        # Extract mid prices for Croissants, Jams, and PB2 if possible.
        # The code checks if these instruments are in order_depths. 
        # If any is missing, we skip the arbitrage logic that tick.
        instruments = ["CROISSANTS", "JAM", "PICNIC_BASKET2"]
        mid_prices = {}
        for inst in instruments:
            if inst not in state.order_depths:
                continue
            od = state.order_depths[inst]
            mid = self.get_mid_price(od)
            mid_prices[inst] = mid
        
        # We can only proceed if we have mid prices for all three.
        if len(mid_prices) < 3:
            return {}, conversions, traderData
        
        croissants_mid = mid_prices["CROISSANTS"]
        jam_mid = mid_prices["JAM"]
        pb2_mid = mid_prices["PICNIC_BASKET2"]
        
        if croissants_mid is None or jam_mid is None or pb2_mid is None:
            return {}, conversions, traderData
        
        # Compute fair value for Picnic Basket 2:
        fair_value = 4 * croissants_mid + 2 * jam_mid
        
        # Define a threshold. If PB2 is off by more than threshold, we trade.
        threshold = 10  # you can tune this based on volatility
        
        # Current PB2 position:
        current_pb2_pos = state.position.get("PICNIC_BASKET2", 0)
        
        # If PB2 is undervalued:
        if pb2_mid + threshold < fair_value:
            # => PB2 is cheap. We want to BUY PB2 (if under limit).
            limit_remaining = self.POSITION_LIMITS["PICNIC_BASKET2"] - current_pb2_pos
            if limit_remaining > 0:
                # Place a buy at or near best ask. 
                # Let's check the best ask from order_depths:
                od_pb2 = state.order_depths["PICNIC_BASKET2"]
                best_ask, best_ask_vol = self.get_best_ask(od_pb2)
                if best_ask is not None and best_ask_vol is not None and best_ask_vol > 0:
                    # For a safe approach, buy only up to min(limit_remaining, best_ask_vol).
                    buy_qty = min(limit_remaining, abs(best_ask_vol))
                    # Place an order
                    print(f"Arb signal: PB2 cheap. Buying {buy_qty} at ask={best_ask}, fair_value={fair_value:.2f}, mid={pb2_mid:.2f}")
                    orders.setdefault("PICNIC_BASKET2", []).append(Order("PICNIC_BASKET2", best_ask, buy_qty))
        
        # If PB2 is overvalued:
        elif pb2_mid - threshold > fair_value:
            # => PB2 is expensive. We want to SELL PB2 if we have room to short
            # (i.e. current position + limit).
            od_pb2 = state.order_depths["PICNIC_BASKET2"]
            best_bid, best_bid_vol = self.get_best_bid(od_pb2)
            if best_bid is not None and best_bid_vol is not None and best_bid_vol > 0:
                # For shorting, let's see how far we can go.
                # The position limit is positive in the dictionary, but we can do shorting
                # up to that limit in negative territory, so the actual capacity is
                # (POSITION_LIMIT + current_pb2_pos).
                short_capacity = current_pb2_pos + self.POSITION_LIMITS["PICNIC_BASKET2"]
                if short_capacity > 0:
                    sell_qty = min(short_capacity, best_bid_vol)
                    print(f"Arb signal: PB2 expensive. Selling {sell_qty} at bid={best_bid}, fair_value={fair_value:.2f}, mid={pb2_mid:.2f}")
                    orders.setdefault("PICNIC_BASKET2", []).append(Order("PICNIC_BASKET2", best_bid, -sell_qty))
        
        return orders, conversions, traderData

    def get_mid_price(self, order_depth: OrderDepth):
        """Compute the mid price from best bid and best ask if available."""
        best_bid, _ = self.get_best_bid(order_depth)
        best_ask, _ = self.get_best_ask(order_depth)
        if best_bid is not None and best_ask is not None:
            return (best_bid + best_ask) / 2
        elif best_bid is not None:
            return best_bid
        elif best_ask is not None:
            return best_ask
        return None

    def get_best_bid(self, od: OrderDepth):
        if od.buy_orders:
            best_bid = max(od.buy_orders.keys())
            return best_bid, abs(od.buy_orders[best_bid])
        return None, None

    def get_best_ask(self, od: OrderDepth):
        if od.sell_orders:
            best_ask = min(od.sell_orders.keys())
            return best_ask, abs(od.sell_orders[best_ask])
        return None, None

ModuleNotFoundError: No module named 'datamodel'